<a href="https://colab.research.google.com/github/abhranil-datascience/SOM_And_AutoEncodersPOC/blob/master/Autoencoders_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################## Mount Drive ######################################## 
#from google.colab import drive
#drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/MLandDLFullCourse/DL/Autoencoders')
########## Suppress Warnings ############
import warnings
warnings.filterwarnings('ignore')
########## Import Dataset ############
import pandas as pd
TrainingSet=pd.read_csv(filepath_or_buffer='u1.base',header=None,delimiter='\t')
TrainingSet=TrainingSet.iloc[:,0:3].values
TestSet=pd.read_csv(filepath_or_buffer='u1.test',header=None,delimiter='\t')
TestSet=TestSet.iloc[:,0:3].values
######### Check Max Movie number and Customer Number ################
### Customer ###
MaxCusNum=max(max(TrainingSet[:,0]),max(TestSet[:,0]))
### Movie ###
MaxMovNum=max(max(TrainingSet[:,1]),max(TestSet[:,1]))
######## Create Dataset ##############
def CreateDataset(Set):
  MovieAndRating=[]
  for count in range(0,MaxCusNum):
    Rating=Set[:,1:3][Set[:,0]==count+1]
    UserRatings=[0]*MaxMovNum
    for i in range(0,MaxMovNum):
      if i+1 in Rating[:,0]:
        tempvar=Rating[:,1][Rating[:,0]==i+1]
        UserRatings[i]=UserRatings[i]+tempvar[0]
    MovieAndRating.append(UserRatings)
  return MovieAndRating
import numpy as np
MovieAndRatingTrain=np.array(CreateDataset(TrainingSet))
MovieAndRatingTest=np.array(CreateDataset(TestSet))
MovieAndRatingTrainAutoencoder=np.expand_dims(MovieAndRatingTrain,axis=0)
MovieAndRatingTestAutoencoder=np.expand_dims(MovieAndRatingTest,axis=0)
############ Implement Autoencoder ####################
from keras.layers import Dense,Input
from keras.models import Model
input_data=Input(shape=(MovieAndRatingTest.shape[0],MovieAndRatingTest.shape[1]))
############################# Encoder #####################################
##### Input Layer #######
encoded=Dense(units=840,activation='relu')(input_data)
##### Layer-1 #######
encoded=Dense(units=420,activation='relu')(encoded)
##### Layer-2 #######
encoded=Dense(units=210,activation='relu')(encoded)
##### Layer-3 #######
encoded=Dense(units=105,activation='relu')(encoded)
############################# Decoder #####################################
##### Layer-1 #######
decoded=Dense(units=105,activation='relu')(encoded)
##### Layer-2 #######
decoded=Dense(units=210,activation='relu')(decoded)
##### Layer-3 #######
decoded=Dense(units=420,activation='relu')(decoded)
##### Layer-4 #######
decoded=Dense(units=420,activation='relu')(decoded)
###### Output Layer ##############
decoded=Dense(units=MovieAndRatingTest.shape[1])(decoded)
#### Create Model #########
autoencoder=Model(input_data,decoded)
autoencoder.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
#### Train Autoencoder ######
autoencoder.fit(x=MovieAndRatingTrainAutoencoder,y=MovieAndRatingTrainAutoencoder,batch_size=8,epochs=1500)
Prediction=autoencoder.predict(MovieAndRatingTestAutoencoder)
Prediction=Prediction[0,:,:]
pred=Prediction.astype(int)
original=MovieAndRatingTest.astype(int)
np.savetxt(fname="OriginalTestSet.csv",delimiter='\t',X=original,fmt='%d')
np.savetxt(fname="PredictedTestSet.csv",delimiter='\t',X=pred,fmt='%d')